## Kubernetes Clusterwide SSL Demo

### Setup

Create a Kind cluster by using an ansible playbook from the project `ansible` folder.


In [ ]:
!cd ../ansible && ansible-playbook playbooks/kind-cluster.yaml

In [ ]:
!cd ../ansible && ansible-playbook playbooks/setup-ecosystem.yaml

In [1]:
!helm upgrade --install seldon-core-v2-crds  ../k8s/helm-charts/seldon-core-v2-crds -n seldon-mesh

Release "seldon-core-v2-crds" has been upgraded. Happy Helming!
NAME: seldon-core-v2-crds
LAST DEPLOYED: Thu May 18 10:27:38 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 2
TEST SUITE: None


In [2]:
!kubectl create namespace ns1

Error from server (AlreadyExists): namespaces "ns1" already exists


In [3]:
!kubectl create namespace ns2

Error from server (AlreadyExists): namespaces "ns2" already exists


In [4]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns1

NAME: seldon-v2-certs
LAST DEPLOYED: Thu May 18 10:28:31 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [5]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns2

NAME: seldon-v2-certs
LAST DEPLOYED: Thu May 18 10:28:32 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [6]:
!cat ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

security:
  controlplane:
    protocol: SSL
    ssl:
      server:
        secret: seldon-controlplane-server
        clientValidationSecret: seldon-controlplane-client
      client:
        secret: seldon-controlplane-client
        serverValidationSecret: seldon-controlplane-server
  envoy:
    protocol: SSL
    ssl:
      upstream:
        server:
          secret: seldon-upstream-server
          clientValidationSecret: seldon-upstream-client
        client:
          secret: seldon-upstream-client
          serverValidationSecret: seldon-upstream-server
      downstream:
        client:
          serverValidationSecret: seldon-downstream-server
        server: 
          secret: seldon-downstream-server


In [7]:
!helm install seldon-v2 ../k8s/helm-charts/seldon-core-v2-setup/ -n seldon-mesh --set controller.clusterwide=true --values ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

NAME: seldon-v2
LAST DEPLOYED: Thu May 18 10:28:47 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [8]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns1

NAME: seldon-v2-runtime
LAST DEPLOYED: Thu May 18 10:30:19 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [9]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns2

NAME: seldon-v2-runtime
LAST DEPLOYED: Thu May 18 10:30:21 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [10]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns1

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


In [11]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns2

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


### Setup TLS Config

In [12]:
MESH_IP=!kubectl get svc seldon-mesh -n ns1 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS1=MESH_IP[0]
import os
os.environ['MESH_IP_NS1'] = MESH_IP_NS1
MESH_IP_NS1

'172.21.255.2'

In [13]:
MESH_IP=!kubectl get svc seldon-mesh -n ns2 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS2=MESH_IP[0]
import os
os.environ['MESH_IP_NS2'] = MESH_IP_NS2
MESH_IP_NS2

'172.21.255.4'

In [14]:
!seldon config add tls ${PWD}/config-dataplane-tls.json

In [15]:
!seldon config activate tls

### Launch model in namespace ns1

In [16]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io/iris created


In [17]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns1

model.mlops.seldon.io/iris condition met


In [18]:
!seldon model infer iris --inference-host ${MESH_IP_NS1}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "2433e5d7-d5b9-493c-b4ec-931418c0e45c",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				2
			]
		}
	]
}


### Launch model in namespace ns2

In [19]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns2

model.mlops.seldon.io/iris created


In [20]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns2

model.mlops.seldon.io/iris condition met


In [21]:
!seldon model infer iris --inference-host ${MESH_IP_NS2}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "e984973e-ed7d-4e9a-83c6-ccfb2217284c",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				2
			]
		}
	]
}


## TearDown

In [22]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io "iris" deleted


In [23]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns2

model.mlops.seldon.io "iris" deleted


In [24]:
!helm delete seldon-v2-runtime -n ns1
!helm delete seldon-v2-certs -n ns1

release "seldon-v2-runtime" uninstalled
release "seldon-v2-certs" uninstalled


In [25]:
!helm delete seldon-v2-runtime -n ns2
!helm delete seldon-v2-certs -n ns2

release "seldon-v2-runtime" uninstalled
release "seldon-v2-certs" uninstalled


In [26]:
!helm delete seldon-v2  -n seldon-mesh

release "seldon-v2" uninstalled
